In [1]:
import fastai
import pandas as pd
import torch

from Quick.cleaning.loading import (
    examine_dataset,
    remove_infs_and_nans
)

from Quick.cleaning.utils import (
    get_file_path
)

from Quick.runners.deep import (
    run_deep_nn_experiment
)

from Quick.runners.residual import (
    run_residual_deep_nn_experiment
)

from Quick.runners.sk import (
    run_sk_experiment
)

from Quick.runners.torch import (
    run_torch_nn_experiment
)

from Quick.constants import *

In [2]:
DATASET


'Dataset'

In [3]:
data_path_1: str = './data/03-11/'
data_sets_1: list = [
    # 'Darknet_experiments_base.csv'
    'Portmap.csv',
    'NetBIOS.csv',
    'LDAP.csv',
    'MSSQL.csv',
    'UDP.csv',
    'UDPLag.csv',
    'Syn.csv',
]

file_path_1: callable = get_file_path(data_path_1)
file_set_1: list = list(map(file_path_1, data_sets_1))
current_job: int = 0

dataset_1 = examine_dataset(0, file_set_1, data_sets_1)

Dataset 0/7: We now look at ./data/03-11/Syn.csv


Loading Dataset: ./data/03-11/Syn.csv
	To Dataset Cache: ./cache/Syn.csv.pickle


        File:				./data/03-11/Syn.csv  
        Job Number:			0
        Shape:				(4320541, 88)
        Samples:			4320541 
        Features:			88
    


In [4]:
# dataset_1 = examine_dataset(0, file_set_1, data_sets_1)
df = remove_infs_and_nans(dataset_1)

deleting 282982 rows with Infinity in column Flow Bytes/s


In [5]:
test_df = dataset_1['Dataset'].head(100) # pandas dataframe
test_df

,Unnamed: 0,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,SimillarHTTP,Inbound,Label
0,445444,172.16.0.5-192.168.50.4-9429-9429-6,172.16.0.5,9429,192.168.50.4,9429,6,2018-11-03 11:36:28.607338,36063894,7,...,29.444864,52.0,1.0,1.202128e+07,6.253623e+06,18628035.0,6193840.0,0,1,Syn
1,113842,172.16.0.5-192.168.50.4-60224-60224-6,172.16.0.5,60224,192.168.50.4,60224,6,2018-11-03 11:36:28.607339,44851366,8,...,0.000000,1.0,1.0,2.066268e+07,1.169783e+07,28934293.0,12391060.0,0,1,Syn
2,176377,172.16.0.5-192.168.50.4-33827-11746-6,192.168.50.4,11746,172.16.0.5,33827,6,2018-11-03 11:36:28.607388,1,2,...,0.000000,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,0,Syn
3,24777,172.16.0.5-192.168.50.4-33828-1431-6,172.16.0.5,33828,192.168.50.4,1431,6,2018-11-03 11:36:28.607391,0,2,...,0.000000,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,1,Syn
4,85100,172.16.0.5-192.168.50.4-5311-5311-6,172.16.0.5,5311,192.168.50.4,5311,6,2018-11-03 11:36:28.607442,35731470,8,...,33.234019,48.0,1.0,1.191047e+07,1.849493e+06,13693985.0,10001398.0,0,1,Syn
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,103728,172.16.0.5-192.168.50.4-31989-31989-6,172.16.0.5,31989,192.168.50.4,31989,6,2018-11-03 11:36:28.614223,41581096,10,...,47.626148,100.0,1.0,1.039524e+07,4.141478e+06,15751901.0,6486134.0,0,1,Syn
96,45707,172.16.0.5-192.168.50.4-33885-46635-6,172.16.0.5,33885,192.168.50.4,46635,6,2018-11-03 11:36:28.614272,71,2,...,0.000000,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,1,Syn
97,306981,172.16.0.5-192.168.50.4-33886-40798-6,172.16.0.5,33886,192.168.50.4,40798,6,2018-11-03 11:36:28.614273,120,2,...,0.000000,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,1,Syn
98,450731,172.16.0.5-192.168.50.4-57073-57073-6,172.16.0.5,57073,192.168.50.4,57073,6,2018-11-03 11:36:28.614341,38988456,10,...,29.781146,124.0,54.0,9.747006e+06,4.512590e+06,16500417.0,7067469.0,0,1,Syn


In [6]:
raw_col_names = test_df.columns

# we trim the column titles to remove leading and trailing whitespace
col_names = list(map(lambda x: x.strip(), raw_col_names))
test_df.columns = col_names

In [7]:
timestamps = test_df['Timestamp']

In [8]:
timestamps: pd.DataFrame = timestamps.copy()

In [9]:
# now we split the datetime column into two columns, one for date and one for time
timestamps: pd.DataFrame = timestamps.str.split(' ', expand=True)
timestamps

,0,1
0,2018-11-03,11:36:28.607338
1,2018-11-03,11:36:28.607339
2,2018-11-03,11:36:28.607388
3,2018-11-03,11:36:28.607391
4,2018-11-03,11:36:28.607442
...,...,...
95,2018-11-03,11:36:28.614223
96,2018-11-03,11:36:28.614272
97,2018-11-03,11:36:28.614273
98,2018-11-03,11:36:28.614341


In [10]:
tdf = test_df.copy()
tdf['Time'] = timestamps[1]
tdf['Date'] = timestamps[0]

In [13]:
tdf

,Unnamed: 0,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,...,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,SimillarHTTP,Inbound,Label,Time,Date
0,445444,172.16.0.5-192.168.50.4-9429-9429-6,172.16.0.5,9429,192.168.50.4,9429,6,2018-11-03 11:36:28.607338,36063894,7,...,1.0,1.202128e+07,6.253623e+06,18628035.0,6193840.0,0,1,Syn,11:36:28.607338,2018-11-03
1,113842,172.16.0.5-192.168.50.4-60224-60224-6,172.16.0.5,60224,192.168.50.4,60224,6,2018-11-03 11:36:28.607339,44851366,8,...,1.0,2.066268e+07,1.169783e+07,28934293.0,12391060.0,0,1,Syn,11:36:28.607339,2018-11-03
2,176377,172.16.0.5-192.168.50.4-33827-11746-6,192.168.50.4,11746,172.16.0.5,33827,6,2018-11-03 11:36:28.607388,1,2,...,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,0,Syn,11:36:28.607388,2018-11-03
3,24777,172.16.0.5-192.168.50.4-33828-1431-6,172.16.0.5,33828,192.168.50.4,1431,6,2018-11-03 11:36:28.607391,0,2,...,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,1,Syn,11:36:28.607391,2018-11-03
4,85100,172.16.0.5-192.168.50.4-5311-5311-6,172.16.0.5,5311,192.168.50.4,5311,6,2018-11-03 11:36:28.607442,35731470,8,...,1.0,1.191047e+07,1.849493e+06,13693985.0,10001398.0,0,1,Syn,11:36:28.607442,2018-11-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,103728,172.16.0.5-192.168.50.4-31989-31989-6,172.16.0.5,31989,192.168.50.4,31989,6,2018-11-03 11:36:28.614223,41581096,10,...,1.0,1.039524e+07,4.141478e+06,15751901.0,6486134.0,0,1,Syn,11:36:28.614223,2018-11-03
96,45707,172.16.0.5-192.168.50.4-33885-46635-6,172.16.0.5,33885,192.168.50.4,46635,6,2018-11-03 11:36:28.614272,71,2,...,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,1,Syn,11:36:28.614272,2018-11-03
97,306981,172.16.0.5-192.168.50.4-33886-40798-6,172.16.0.5,33886,192.168.50.4,40798,6,2018-11-03 11:36:28.614273,120,2,...,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,1,Syn,11:36:28.614273,2018-11-03
98,450731,172.16.0.5-192.168.50.4-57073-57073-6,172.16.0.5,57073,192.168.50.4,57073,6,2018-11-03 11:36:28.614341,38988456,10,...,54.0,9.747006e+06,4.512590e+06,16500417.0,7067469.0,0,1,Syn,11:36:28.614341,2018-11-03


In [ ]:
# now we sort the data by the timestamp
timestamps = timestamps.sort_values(by=1)
timestamps

In [4]:
df.columns = list(map(lambda x: x.strip(), df.columns))

In [5]:
df = df.sort_values(by='Timestamp')

In [20]:
df.tail(20)

,Unnamed: 0,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,SimillarHTTP,Inbound,Label
512563,6839,173.194.206.109-192.168.50.6-465-58745-6,173.194.206.109,465,192.168.50.6,58745,6,2018-11-03 17:35:25.466094,219,1,...,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,1,BENIGN
512564,8831,173.194.206.109-192.168.50.6-465-58745-6,192.168.50.6,58745,173.194.206.109,465,6,2018-11-03 17:35:25.466313,52814,1,...,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,0,BENIGN
512565,7603,172.16.0.5-192.168.50.4-59227-33367-6,172.16.0.5,59227,192.168.50.4,33367,6,2018-11-03 17:35:31.769315,160059,4,...,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,1,Syn
512566,7471,192.168.10.50-192.168.50.6-21-58746-6,192.168.50.6,58746,192.168.10.50,21,6,2018-11-03 17:35:35.843877,8999770,6,...,0.000000e+00,2999280.0,2999280.0,6.000488e+06,0.000000e+00,6000488.0,6000488.0,0,1,BENIGN
512567,12709,172.16.0.5-192.168.50.4-0-0-0,172.16.0.5,0,192.168.50.4,0,0,2018-11-03 17:35:37.227169,79122152,30,...,1.968234e+06,5388134.0,110.0,9.581547e+06,4.582314e+06,16183731.0,5294188.0,0,1,Syn
512568,7391,172.16.0.5-192.168.50.4-44423-4018-6,172.16.0.5,44423,192.168.50.4,4018,6,2018-11-03 17:35:47.968404,129284,4,...,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,1,Syn
512569,12826,192.168.50.6-4.2.2.4-51890-53-17,192.168.50.6,51890,4.2.2.4,53,17,2018-11-03 17:35:54.482705,22630,2,...,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,0,BENIGN
512570,8732,192.168.50.6-8.8.8.8-51890-53-17,192.168.50.6,51890,8.8.8.8,53,17,2018-11-03 17:35:54.502714,115945,2,...,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,0,BENIGN
512571,11374,192.168.50.6-65.55.252.93-58747-443-6,192.168.50.6,58747,65.55.252.93,443,6,2018-11-03 17:35:54.506188,9012862,6,...,0.000000e+00,3012470.0,3012470.0,6.000389e+06,0.000000e+00,6000389.0,6000389.0,0,0,BENIGN
512572,10138,172.16.0.5-192.168.50.4-45023-445-6,172.16.0.5,45023,192.168.50.4,445,6,2018-11-03 17:35:59.549602,540751,4,...,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,1,Syn


In [6]:
# lets examine how many unique values are in each label
df['Label'].value_counts()

Syn       4002078
BENIGN      35387
Name: Label, dtype: int64

In [7]:
benign_df = df[df['Label'] == 'BENIGN']
benign_df.shape

(35387, 88)

In [8]:
syn_df: pd.DataFrame = df[df['Label'] == 'Syn'].sample(benign_df.shape[0], random_state=42)
syn_df.shape

(35387, 88)

In [9]:
ddos_syn_df = pd.concat([benign_df, syn_df])

In [10]:
list(ddos_syn_df.columns)

['Unnamed: 0',
 'Flow ID',
 'Source IP',
 'Source Port',
 'Destination IP',
 'Destination Port',
 'Protocol',
 'Timestamp',
 'Flow Duration',
 'Total Fwd Packets',
 'Total Backward Packets',
 'Total Length of Fwd Packets',
 'Total Length of Bwd Packets',
 'Fwd Packet Length Max',
 'Fwd Packet Length Min',
 'Fwd Packet Length Mean',
 'Fwd Packet Length Std',
 'Bwd Packet Length Max',
 'Bwd Packet Length Min',
 'Bwd Packet Length Mean',
 'Bwd Packet Length Std',
 'Flow Bytes/s',
 'Flow Packets/s',
 'Flow IAT Mean',
 'Flow IAT Std',
 'Flow IAT Max',
 'Flow IAT Min',
 'Fwd IAT Total',
 'Fwd IAT Mean',
 'Fwd IAT Std',
 'Fwd IAT Max',
 'Fwd IAT Min',
 'Bwd IAT Total',
 'Bwd IAT Mean',
 'Bwd IAT Std',
 'Bwd IAT Max',
 'Bwd IAT Min',
 'Fwd PSH Flags',
 'Bwd PSH Flags',
 'Fwd URG Flags',
 'Bwd URG Flags',
 'Fwd Header Length',
 'Bwd Header Length',
 'Fwd Packets/s',
 'Bwd Packets/s',
 'Min Packet Length',
 'Max Packet Length',
 'Packet Length Mean',
 'Packet Length Std',
 'Packet Length Varianc

In [11]:

ndf: pd.DataFrame = ddos_syn_df.copy().drop(
    [
        'Unnamed: 0',
        'Flow ID',
        'Source IP',
        'Source Port',
        'Destination IP',
        'Destination Port',
        'Timestamp',
        'SimillarHTTP'
    ], 
    axis=1)


In [12]:
ddos_syn_df.columns

Index(['Unnamed: 0', 'Flow ID', 'Source IP', 'Source Port', 'Destination IP',
       'Destination Port', 'Protocol', 'Timestamp', 'Flow Duration',
       'Total Fwd Packets', 'Total Backward Packets',
       'Total Length of Fwd Packets', 'Total Length of Bwd Packets',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Min Packet Length', 'Max Packet Length', 'Packet Le

In [13]:
ndf.columns

Index(['Protocol', 'Flow Duration', 'Total Fwd Packets',
       'Total Backward Packets', 'Total Length of Fwd Packets',
       'Total Length of Bwd Packets', 'Fwd Packet Length Max',
       'Fwd Packet Length Min', 'Fwd Packet Length Mean',
       'Fwd Packet Length Std', 'Bwd Packet Length Max',
       'Bwd Packet Length Min', 'Bwd Packet Length Mean',
       'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s',
       'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min',
       'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max',
       'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std',
       'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags',
       'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Min Packet Length', 'Max Packet Length', 'Packet Length Mean',
       'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
       'SYN Flag Count', 'R

In [14]:
report: pd.DataFrame = dataset_1['Feature_stats']
# we check how many unique values are in each column
# df['Label'].value_counts()
report.columns = list(map(lambda x: x.strip(), report.columns))
num_samples = dataset_1['Dataset'].shape[0]
r2 = report.drop(['Dataset', 'Value'], axis=1)
# report
r2

droplist = []

for col in r2.columns:
    if (r2[col][2] == num_samples):
        droplist.append(col)

droplist


['Bwd PSH Flags',
 'Fwd URG Flags',
 'Bwd URG Flags',
 'FIN Flag Count',
 'PSH Flag Count',
 'ECE Flag Count',
 'Fwd Avg Bytes/Bulk',
 'Fwd Avg Packets/Bulk',
 'Fwd Avg Bulk Rate',
 'Bwd Avg Bytes/Bulk',
 'Bwd Avg Packets/Bulk',
 'Bwd Avg Bulk Rate']

In [15]:
ndf_2 = ndf.drop(droplist, axis=1)

In [16]:
ndf_2.shape

(70774, 68)

In [ ]:
report

In [ ]:
report[0].value_counts()

In [ ]:
import importlib
import Quick.runners.sk
importlib.reload(Quick.runners.sk)

In [ ]:
for col in ndf.columns:
    # print out the type of each column

    print(f"Column: {col}")
    print(f"Type: {ndf[col].dtype}")

In [17]:
pr = [
    DEFAULT_PROCS[0],
    DEFAULT_PROCS[1],
    DEFAULT_PROCS[2]
]

In [18]:
# we convert all values in ndf_2 to floats
# ndf_2 = ndf_2.astype(float)
for col in ndf_2.columns:
    try:
        ndf_2[col].astype(float)
    except:
        print(col)

Label


In [19]:
import importlib
import Quick.runners.sk
importlib.reload(Quick.runners.sk)
results_sk = Quick.runners.sk.run_sk_experiment(ndf_2, dataset_1['File'], 'Label', procs=pr)


Shape of Input Data: (70774, 68)
hello
categorical features:  1 ['Protocol']
continuous features:  66 ['Flow IAT Max', 'CWE Flag Count', 'Total Backward Packets', 'min_seg_size_forward', 'Active Mean', 'Max Packet Length', 'Flow Bytes/s', 'Subflow Bwd Packets', 'Init_Win_bytes_forward', 'Bwd Packet Length Std', 'Bwd IAT Total', 'Subflow Bwd Bytes', 'Bwd Packets/s', 'Fwd Packet Length Min', 'Fwd IAT Min', 'Flow IAT Mean', 'Packet Length Mean', 'Flow IAT Min', 'Subflow Fwd Packets', 'Fwd Packet Length Std', 'Fwd IAT Total', 'Packet Length Std', 'Total Fwd Packets', 'Packet Length Variance', 'Active Min', 'Fwd IAT Mean', 'Fwd IAT Max', 'Flow IAT Std', 'Down/Up Ratio', 'ACK Flag Count', 'Bwd IAT Min', 'Flow Duration', 'Idle Std', 'Fwd Packet Length Max', 'Idle Max', 'Total Length of Fwd Packets', 'Idle Mean', 'Flow Packets/s', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Init_Win_bytes_backward', 'act_data_pkt_fwd', 'SYN Flag Count', 'Total Length of Bwd Packets', 'Active Std', 'Fwd

In [ ]:
tdf = ndf_2.copy()

In [ ]:
tdf_2 = tdf.drop(['Label'], axis=1)

In [ ]:
tdf_2.shape

In [ ]:
tdf.shape

In [ ]:
import numpy

In [ ]:
array = numpy.asarray(tdf_2.values)